In [208]:
import requests
import time
import numpy as np
import pandas as pd
import copy
from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

# Third environmement

## Get The Data

In [209]:
#attributes fa
USER_ID = 'IAHIZPIW80WPGDW7P7JE'
BASE_URL = "http://35.180.178.243"
url_reset = BASE_URL + "/reset"
url_predict = BASE_URL + "/predict"
params = {"user_id" : USER_ID}

def load_data(url_reset,params) :
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    return data

data = load_data(url_reset,params)
nb_users = data["nb_users"]
nb_items = data["nb_items"]
next_state = data["next_state"]

data.keys()

dict_keys(['action_history', 'nb_items', 'nb_users', 'next_state', 'rewards_history', 'state_history'])

In [210]:
blacklisted_set = set(("next_state","nb_items","nb_users"))
new_dict = {key : value for key, value in data.items() if key not in blacklisted_set}

df = pd.DataFrame(new_dict)
df.head()

,action_history,rewards_history,state_history
0,13,42.678405,"[[32, 0, 561.9608319155392, 2.7606149744183837..."
1,18,342.929576,"[[57, 0, 561.9608319155392, 0.3076770015376174..."
2,15,0.000000,"[[38, 0, 561.9608319155392, 1.7039990882609266..."
3,23,0.000000,"[[79, 0, 561.9608319155392, -0.679139726452107..."
4,6,0.000000,"[[50, 0, 561.9608319155392, 1.3492680753336916..."


In [211]:
df.describe()

,action_history,rewards_history
count,200.000000,200.000000
mean,14.240000,129.973246
std,8.742112,248.991807
min,0.000000,0.000000
25%,7.000000,0.000000
50%,14.000000,0.000000
75%,22.000000,152.174722
max,29.000000,963.357634


In [212]:
df[df['rewards_history']==0].count()

action_history     140
rewards_history    140
state_history      140
dtype: int64

In [213]:
users_items_df = []
for i in range(df.shape[0]):
    users_items_df += df['state_history'][i] 
    
users_items_df = pd.DataFrame(users_items_df,columns = ['user_id','item_id',
                                                        'price','v1','v2','v3','v4','v5'])
users_items_df

,user_id,item_id,price,v1,v2,v3,v4,v5
0,32,0,561.960832,2.760615,1.314982,-0.398634,2.223695,0.835217
1,32,1,237.133114,2.760615,1.314982,2.323786,2.157464,3.102571
2,32,2,825.248994,2.760615,1.314982,2.338465,0.426121,1.230357
3,32,3,963.357634,2.760615,1.314982,1.482943,1.885303,-0.710399
4,32,4,478.175399,2.760615,1.314982,0.779073,0.683714,-0.372410
5,32,5,571.098599,2.760615,1.314982,0.458886,0.750748,3.071613
6,32,6,182.731294,2.760615,1.314982,1.212972,-0.522265,0.400256
7,32,7,473.395808,2.760615,1.314982,0.589554,0.882864,1.326484
8,32,8,141.989198,2.760615,1.314982,1.884052,1.993851,0.674759
9,32,9,720.382024,2.760615,1.314982,1.312128,1.090891,1.812324


In [137]:
# Eliminate the 'v5'
new_users_items_df = users_items_df.drop(columns='v5').drop_duplicates() 

In [9]:
# We can see that the first two covariates variables are features about users 
# and the forth and fifth variable are features about items
# The variable 5 depends on .....

user_v1 = users_items_df['v1'].unique()
user_v2 = users_items_df['v2'].unique()
users_ids = users_items_df['user_id'].unique()
users_content= pd.DataFrame({'user_id' : users_ids,'v1':user_v1,'v2':user_v2 })

item_v3 = users_items_df['v3'].unique()
item_v4 = users_items_df['v4'].unique()
item_id = users_items_df['item_id'].unique()
items_content= pd.DataFrame({'item_id' : item_id,'v3':item_v3,'v4':item_v4 })

In [87]:
# We don't have all the metadat from all the users
users_content['user_id'].unique().shape

(87,)

## Data Preprocessing

In [10]:
pos_df = df[df["rewards_history"] > 0].reset_index(drop=True)
neg_df = df[df["rewards_history"] == 0].reset_index(drop=True)

In [11]:
pos_df["user_id"] = [pos_df["state_history"][i][0][0] for i in range(pos_df.shape[0])]
pos_df["item_id_action"] = [pos_df["state_history"][i][pos_df["action_history"][i]][1] for i in range(pos_df.shape[0])]
pos_df.head()## There are some users that have more than one positive item

,action_history,rewards_history,state_history,user_id,item_id_action
0,7,564.746210,"[[36, 0, 540.5126332718834, 1.782519358910701,...",36,7
1,14,512.183329,"[[22, 0, 540.5126332718834, 0.1186121387056536...",22,14
2,19,738.509790,"[[10, 0, 540.5126332718834, -0.210541174376374...",10,19
3,4,297.761357,"[[4, 0, 540.5126332718834, 1.251532030958378, ...",4,4
4,13,992.227109,"[[68, 0, 540.5126332718834, 0.6795394801339365...",68,13


In [12]:
neg_df["user_id"] = [neg_df["state_history"][i][0][0] for i in range(neg_df.shape[0])]
neg_df["item_id_action"] = [neg_df["state_history"][i][neg_df["action_history"][i]][1] for i in range(neg_df.shape[0])]
neg_df.head()

,action_history,rewards_history,state_history,user_id,item_id_action
0,22,0.0,"[[27, 0, 540.5126332718834, -0.352061653645617...",27,22
1,18,0.0,"[[46, 0, 540.5126332718834, 2.2085491075520194...",46,18
2,17,0.0,"[[51, 0, 540.5126332718834, 1.4813623958600743...",51,17
3,9,0.0,"[[33, 0, 540.5126332718834, -0.023613619655153...",33,9
4,15,0.0,"[[49, 0, 540.5126332718834, 2.9426282310959033...",49,15


# Model

In [37]:
import tensorflow as tf

def identity_loss(y_true, y_pred):
    """Ignore y_true and return the mean of y_pred
    
    This is a hack to work-around the design of the Keras API that is
    not really suited to train networks with a triplet loss by default.
    """
    return tf.reduce_mean(y_pred + 0 * y_true)


def margin_comparator_loss(inputs, margin=1.):
    """Comparator loss for a pair of precomputed similarities
    
    If the inputs are cosine similarities, they each have range in
    (-1, 1), therefore their difference have range in (-2, 2). Using
    a margin of 1. can therefore make sense.

    If the input similarities are not normalized, it can be beneficial
    to use larger values for the margin of the comparator loss.
    """
    positive_pair_sim, negative_pair_sim = inputs
    return tf.maximum(negative_pair_sim - positive_pair_sim + margin, 0)

## Naive Model(The most expensive item)

In [13]:
def naive_model_expensive(state):
    greater_index = 0
    greater_value = 0
    
    for i in range(len(state)):
        if(state[i][2] > greater_value):
            greater_value = state[i][2]
            greater_index = i
    return greater_index  

In [14]:
number_run = 3
generations = 100
total_reward = 0

for k in range(number_run):
    
    for i in range(generations):

        predicted_item = naive_model_expensive(next_state)

        params['recommended_item'] = predicted_item

        r = requests.get(url = url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward
        next_state = r['state']
        
        
print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run,))

Total reward: 94429.421857 and Total reward per run: 31476.473952


## Naive Model(The cheapest item)

In [15]:
def naive_model_cheapest(state):
    cheapest_index = 0
    cheapest_value =  state[0][2]
    
    for i in range(len(state)):
        if(state[i][2] < cheapest_value):
            cheapest_value = state[i][2]
            cheapest_index = i
            
    return cheapest_index  

In [16]:
number_run = 3
generations = 100
total_reward = 0

for k in range(number_run):
    
    for i in range(generations):

        predicted_item = naive_model_cheapest(next_state)

        params['recommended_item'] = predicted_item

        r = requests.get(url = url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward
        next_state = r['state']
        
        
print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run,))

Total reward: 10981.159575 and Total reward per run: 3660.386525


## Model 1

In [17]:
from keras.models import Model
from keras.layers import Embedding, Flatten, Input, Dense
from keras.layers import Lambda, Dot
from keras.regularizers import l2
from keras.layers.merge import dot, concatenate


def build_models(n_users, n_items, latent_dim=64, l2_reg=0):
    """Build a triplet model and its companion similarity model
    
    The triplet model is used to train the weights of the companion
    similarity model. The triplet model takes 1 user, 1 positive item
    (relative to the selected user) and one negative item and is
    trained with comparator loss.
    
    The similarity model takes one user and one item as input and return
    compatibility score (aka the match score).
    """
    # Common architectural components for the two models:
    # - symbolic input placeholders
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')

    # - embeddings
    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(input_dim=n_users + 1, output_dim=latent_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)
    
    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(input_dim=n_items + 1, output_dim=latent_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # - similarity computation between embeddings
    positive_similarity = Dot(name="positive_similarity",
                              axes=1, normalize=True)(
        [user_embedding, positive_item_embedding])
    negative_similarity = Dot(name="negative_similarity",
                              axes=1, normalize=True)(
        [user_embedding, negative_item_embedding])

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss,
                          name='comparator_loss',
                          output_shape=(1,))([positive_similarity, negative_similarity])

    triplet_model = Model(inputs=[user_input,
                                  positive_item_input,
                                  negative_item_input],
                          outputs=triplet_loss)
    
    # The match-score model, only use at inference to rank items for a given
    # model: the model weights are shared with the triplet_model therefore
    # we do not need to train it and therefore we do not need to plug a loss
    # and an optimizer.
    match_model = Model(inputs=[user_input, positive_item_input],
                        outputs=positive_similarity)
    
    return triplet_model, match_model


In [18]:
def sample_triplets(pos_df,max_items):
    """
    Sample negatives at random
    
    """
    range_itens = np.arange(max_items)
    user_ids = pos_df['user_id'].values
    pos_item_ids = pos_df['item_id_action'].values
    
    
    user_pos_items = pos_df.groupby('user_id')['item_id_action'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([item for item in range_itens 
                                       if not item in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens,1)
        neg_item_ids = np.append(neg_item_ids,number)
    
  
    return [user_ids, pos_item_ids, neg_item_ids]

In [19]:
number_run = 3
n_epochs = 30
total_reward = 0
generations = 100
max_items = 30

for k in range(number_run):

    triplet_model, match_model = build_models(nb_users, nb_items, latent_dim=16, l2_reg=1e-6)
    # we plug the identity loss and the a fake target variable ignored by
    # the model to be able to use the Keras API to train the triplet model
    triplet_model.compile(loss=identity_loss, optimizer="adam")
    fake_y = np.ones_like(pos_df['user_id'])
    params = {'user_id': USER_ID}

    positive_data = pos_df.copy()
    
    for i in range(n_epochs):
        # Sample new negatives to build different triplets at each epoch
        triplet_inputs = sample_triplets(positive_data, max_items=nb_items)

        # Fit the model incrementally by doing a single pass over the
        # sampled triplets.
        triplet_model.fit(triplet_inputs, fake_y, shuffle=True, epochs=1)

    for j in range(generations):


        # The user that we need to recommend a product
        new_user_id = next_state[0][0]
        # Take all the items in the new state
        items_ids = np.array([next_state[i][1]
                              for i in range(len(next_state))])

        # Create an array of the same size of 'items_ids' of all values equal to 'new_user_id'
        repeated_user_id = np.empty_like(items_ids)
        repeated_user_id.fill(new_user_id)

        # predict the product using our model
        predicted = match_model.predict([repeated_user_id, items_ids])
        predicted_item = np.argmax(predicted)
        params['recommended_item'] = predicted_item

        # take the reward from the API
        r = requests.get(url=url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward

        # (Online learning)Check if the reward is greater than 0 , if yes train again 
        # the model the sample point
        if(reward > 0):
            range_itens = np.arange(max_items)
            user_pos_items = pos_df.groupby('user_id')['item_id_action'].apply(list)
            if new_user_id in set(user_pos_items.index):
                neg_item_id = np.random.choice([item for item in range_itens 
                                           if not item in user_pos_items[user_pos_items.index == new_user_id]])
            else:
                neg_item_id = np.random.choice(range_itens,1)
            
           
            
            triplet_inputs = [[new_user_id], [predicted_item], [neg_item_id]]
    
            fake_y = np.ones_like([1])
            triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                               batch_size=64, epochs=1)
    
            
        # Take the nex_state
        next_state = r['state']


print("Total reward: %f and Total reward per run: %f" %(total_reward, total_reward/number_run,))

Epoch 1/1
64/64 [==============================] - 0s 6ms/step - loss: 1.0224
Epoch 1/1
64/64 [==============================] - 0s 78us/step - loss: 0.9764
Epoch 1/1
64/64 [==============================] - 0s 66us/step - loss: 0.9556
Epoch 1/1
64/64 [==============================] - 0s 100us/step - loss: 0.8529
Epoch 1/1
64/64 [==============================] - 0s 72us/step - loss: 0.8445
Epoch 1/1
64/64 [==============================] - 0s 82us/step - loss: 0.7553
Epoch 1/1
64/64 [==============================] - 0s 112us/step - loss: 0.7143
Epoch 1/1
64/64 [==============================] - 0s 78us/step - loss: 0.6954
Epoch 1/1
64/64 [==============================] - 0s 87us/step - loss: 0.6616
Epoch 1/1
64/64 [==============================] - 0s 93us/step - loss: 0.6529
Epoch 1/1
64/64 [==============================] - 0s 69us/step - loss: 0.6286
Epoch 1/1
64/64 [==============================] - 0s 62us/step - loss: 0.5228
Epoch 1/1
64/64 [==============================] - 

Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.1844
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.9743
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 1.1652
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.3372
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 1.0656
Epoch 1/1
64/64 [==============================] - 0s 7ms/step - loss: 0.9897
Epoch 1/1
64/64 [==============================] - 0s 71us/step - loss: 0.9428
Epoch 1/1
64/64 [==============================] - 0s 61us/step - loss: 0.8774
Epoch 1/1
64/64 [==============================] - 0s 143us/step - loss: 0.8242
Epoch 1/1
64/64 [==============================] - 0s 64us/step - loss: 0.8009
Epoch 1/1
64/64 [==============================] - 0s 90us/step - loss: 0.7505
Epoch 1/1
64/64 [==============================] - 0s 79us/step - loss

## Model 2

In [116]:
from keras.models import Model, Sequential
from keras.layers import Embedding, Flatten, Input, Dense, Dropout
from keras.layers import Concatenate, Lambda
from keras.regularizers import l2


def make_interaction_mlp(input_dim, n_hidden=1, hidden_size=64,
                         dropout=0, l2_reg=None):
    """
    Build the shared multi layer perceptron

    """
    mlp = Sequential()
    if n_hidden == 0:
        # Plug the output unit directly: this is a simple
        # linear regression model. Not dropout required.
        mlp.add(Dense(1, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
    else:
        mlp.add(Dense(hidden_size, input_dim=input_dim,
                      activation='relu', kernel_regularizer=l2_reg))
        mlp.add(Dropout(dropout))
        for i in range(n_hidden - 1):
            mlp.add(Dense(hidden_size, activation='relu',
                          kernel_regularizer=l2_reg))
            mlp.add(Dropout(dropout))
        mlp.add(Dense(1, activation='relu', kernel_regularizer=l2_reg))
    return mlp


def build_models(n_users, n_items, user_dim=32, item_dim=64,
                 n_hidden=1, hidden_size=64, dropout=0, l2_reg=0):
    """
    Build models to train a deep triplet network

    """
    user_input = Input((1,), name='user_input')
    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    positive_meta_input = Input((2,), name='positive_meta_input')
    negative_meta_input = Input((2,), name='negative_meta_input')

    l2_reg = None if l2_reg == 0 else l2(l2_reg)
    user_layer = Embedding(n_users + 1, user_dim, input_length=1,
                           name='user_embedding', embeddings_regularizer=l2_reg)

    # The following embedding parameters will be shared to encode both
    # the positive and negative items.
    item_layer = Embedding(n_items + 1, item_dim, input_length=1,
                           name="item_embedding", embeddings_regularizer=l2_reg)

    user_embedding = Flatten()(user_layer(user_input))
    positive_item_embedding = Flatten()(item_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_layer(negative_item_input))

    # Similarity computation between embeddings using a MLP similarity
    positive_embeddings_pair = Concatenate(name="positive_embeddings_pair")(
        [user_embedding, positive_item_embedding, positive_meta_input])
    positive_embeddings_pair = Dropout(dropout)(positive_embeddings_pair)

    negative_embeddings_pair = Concatenate(name="negative_embeddings_pair")(
        [user_embedding, negative_item_embedding, negative_meta_input])
    negative_embeddings_pair = Dropout(dropout)(negative_embeddings_pair)

    # Instanciate the shared similarity architecture
    interaction_layers = make_interaction_mlp(
        user_dim + item_dim + 2, n_hidden=n_hidden, hidden_size=hidden_size,
        dropout=dropout, l2_reg=l2_reg)

    positive_similarity = interaction_layers(positive_embeddings_pair)
    negative_similarity = interaction_layers(negative_embeddings_pair)

    # The triplet network model, only used for training
    triplet_loss = Lambda(margin_comparator_loss, output_shape=(1,),
                          name='comparator_loss')(
        [positive_similarity, negative_similarity])

    deep_triplet_model = Model(inputs=[user_input,
                                       positive_item_input,
                                       negative_item_input,
                                       positive_meta_input,
                                       negative_meta_input
                                       ],
                               outputs=[triplet_loss])

    # The match-score model, only used at inference
    deep_match_model = Model(inputs=[user_input, positive_item_input, positive_meta_input],
                             outputs=[positive_similarity])

    return deep_match_model, deep_triplet_model

In [117]:
def sample_triplets(pos_df, items_content, max_items):
    """
    Sample the data and put in the format 
    [user_ids, pos_item_ids, neg_item_ids,pos_item_metadata,neg_item_metadata]

    """
    range_itens = np.arange(max_items)
    user_ids = pos_df['user_id'].values
    

    pos_item_ids = pos_df['item_id_action'].values
    pos_item_metadata = items_content.loc[pos_item_ids][['v3', 'v4']].values

    
    user_pos_items = pos_df.groupby('user_id')['item_id_action'].apply(list)
    neg_item_ids = np.array([])
    for i in user_ids:
        if i in set(user_pos_items.index):
            number = np.random.choice([item for item in range_itens 
                                       if not item in user_pos_items[user_pos_items.index == i]])
        else:
            number = np.random.choice(range_itens, 1)
        neg_item_ids = np.append(neg_item_ids, number)

    neg_item_metadata = items_content.loc[neg_item_ids][['v3', 'v4']].values

    return [user_ids, pos_item_ids, neg_item_ids, pos_item_metadata, neg_item_metadata]

In [118]:
hyper_parameters = dict(
    user_dim=50,
    item_dim=15,
    n_hidden=1,
    hidden_size=16,
    dropout=0.5,
    l2_reg=0
)

In [196]:
number_run = 3
n_epochs = 30
total_reward = 0
generations = 100
max_items = 30 

for k in range(number_run):

    deep_match_model, deep_triplet_model = build_models(nb_users, nb_items,**hyper_parameters)
    # we plug the identity loss and the a fake target variable ignored by
    # the model to be able to use the Keras API to train the triplet model
    deep_triplet_model.compile(loss=identity_loss, optimizer="adam")
    fake_y = np.ones_like(pos_df['user_id'])
    params = {'user_id': USER_ID}
    
    positive_data = pos_df.copy()
    
    # Training the model 
    for i in range(n_epochs):
        # Sample new negatives to build different triplets at each epoch
        triplet_inputs = sample_triplets(positive_data,items_content, max_items=nb_items)

        # Fit the model incrementally by doing a single pass over the
        # sampled triplets.
        deep_triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                               batch_size=64, epochs=1)


    for j in range(generations):
        
        # Predicting
        new_user_id = next_state[0][0]
        items_ids = np.array([next_state[i][1] for i in range(len(next_state))])
        repeated_user_id = np.empty_like(items_ids)
        repeated_user_id.fill(new_user_id)

        pos_metadata = items_content.loc[items_ids][['v3', 'v4']].values
        predicted = deep_match_model.predict([repeated_user_id, items_ids, pos_metadata])
        predicted_item = np.argmax(predicted)
        params['recommended_item'] = predicted_item

        r = requests.get(url=url_predict, params=params).json()
        reward = r['reward']
        total_reward += reward

        # Online learning
        if(reward > 0):
            range_itens = np.arange(max_items)
            user_pos_items = positive_data.groupby('user_id')['item_id_action'].apply(list)
            if new_user_id in set(user_pos_items.index):
                neg_item_id = np.random.choice([item for item in range_itens 
                                           if not item in user_pos_items[user_pos_items.index == new_user_id]])
            else:
                neg_item_id = np.random.choice(range_itens,1)
            
            pos_item_metadata = items_content.set_index('item_id').loc[predicted_item].values
            neg_item_metadata = items_content.set_index('item_id').loc[neg_item_id].values
            
            triplet_inputs = [[new_user_id], [predicted_item], [neg_item_id], 
                              pos_item_metadata.reshape((1,2)), neg_item_metadata.reshape((1,2))]
    
            fake_y = np.ones_like([1])
            deep_triplet_model.fit(triplet_inputs, fake_y, shuffle=True,
                               batch_size = 8, epochs=1)

        next_state = r['state']

print("Total reward: %f and Total reward per run: %f"%(total_reward, total_reward/number_run))

KeyError: 'user_id'

## Model 3